In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing

import os, shutil # sorting files
import pathlib

import matplotlib.pyplot as plt # data visualization
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

tf.__version__

In [ ]:
batch_size = 8
img_height = 224
img_width = 224

from tensorflow.keras.preprocessing import image_dataset_from_directory

train_ds = image_dataset_from_directory(
  '../input/happywhalesortedbyindividual/train_species_list',
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  label_mode = 'categorical',
  labels='inferred',
  interpolation='nearest',
  shuffle = True)

class_names = train_ds.class_names

test_ds = image_dataset_from_directory(
  '../input/jpeg-happywhale-256x256/test_images-256-256',
  image_size=(img_height, img_width),
  batch_size=32,
  interpolation='nearest',
  shuffle = False,
  label_mode = None)

def convert_to_float(image):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image

In [ ]:
model = tf.keras.models.load_model("../input/vgg16trainedhwmodelv1/vgg-updated-1.h5")

In [ ]:
img_df = pd.read_csv('../input/happy-whale-and-dolphin/sample_submission.csv')
img_list = img_df['image'].tolist()
#probabilities = model.predict(test_ds)

In [ ]:
from tensorflow.keras.preprocessing import image

img_path = '../input/jpeg-happywhale-256x256/test_images-256-256/test_images-256-256/'

predictions = {}
for i in range(len(img_list)):
    img = image.load_img(img_path + img_list[i], target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = tf.image.convert_image_dtype(img_array, dtype=tf.float32)
    img_batch = np.expand_dims(img_array, axis=0)
    
    probabilities = model.predict(img_batch)
    class_prob = probabilities[0, :]
    top_values_index = sorted(range(len(class_prob)), key=lambda j: class_prob[j])[-5:]
    
    predictions[img_list[i]] = class_names[top_values_index[0]] + ' new_individual ' + class_names[top_values_index[1]] + ' ' + class_names[top_values_index[2]] + ' ' + class_names[top_values_index[3]] + ' ' + class_names[top_values_index[4]]
    

In [ ]:
#predictions = {}
#for j in range(len(probabilities)):
#    class_prob = probabilities[j, :]
#    top_values_index = sorted(range(len(class_prob)), key=lambda i: class_prob[i])[-5:]
#    predictions[img_list[j]] = class_names[top_values_index[0]] + ' new_individual ' + class_names[top_values_index[1]] + ' ' + class_names[top_values_index[2]] + ' ' + class_names[top_values_index[3]] + ' ' + class_names[top_values_index[4]]
#    if(j%3000 == 0):
#        print(j)
#print(predictions)

In [ ]:
pred_df = pd.DataFrame(predictions.items(), columns=['image', 'predictions'])
pred_df.to_csv('submission.csv', index=False)